# Cleaning

In [1]:
!pip install contractions
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 4.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 11.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=b53b03c18005943f3f2b560e80ba68197f0191006da32934d2aac345a1245dc6
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import os
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from langdetect import detect

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
run_mode = 'kaggle'
if run_mode == 'kaggle':
    train = pd.read_csv('/kaggle/input/fake-news/train.csv')
    test = pd.read_csv('/kaggle/input/fake-news/test.csv')
else: # local
    train = pd.read_csv('../train.csv')
    test = pd.read_csv('../test.csv')
train.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


Remove the author column

In [5]:
train.drop(columns=['author'], inplace=True)
train.head()

,id,title,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
train.head()

,id,title,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Print \nAn Iranian woman has been sentenced to...,1


Null values

In [7]:
print(train['label'].value_counts())

label
1    10413
0    10387
Name: count, dtype: int64


In [8]:
train.isnull().sum()

id         0
title    558
text      39
label      0
dtype: int64

In [9]:
train = train.dropna(subset=['text'])

Outliers

In [10]:
train[train['text'].str.len() <= 2].head()

,id,title,text,label
82,82,Huma’s Weiner Dogs Hillary,,1
169,169,Mohamad Khweis: Another “Virginia Man” (Palest...,,1
295,295,A Connecticut Reader Reports Record Voter Regi...,,1
470,470,BULLETIN: There ARE Righteous Jews For Trump!;...,,1
592,592,Is your promising internet career over now Vin...,,1


In [11]:
train = train[train['text'].str.len() > 2]

In [12]:
train[train['text'] == "source Add To The Conversation Using Facebook Comments"]['text']

519      source Add To The Conversation Using Facebook ...
3206     source Add To The Conversation Using Facebook ...
4726     source Add To The Conversation Using Facebook ...
4781     source Add To The Conversation Using Facebook ...
5052     source Add To The Conversation Using Facebook ...
5539     source Add To The Conversation Using Facebook ...
6891     source Add To The Conversation Using Facebook ...
7055     source Add To The Conversation Using Facebook ...
7628     source Add To The Conversation Using Facebook ...
7946     source Add To The Conversation Using Facebook ...
8386     source Add To The Conversation Using Facebook ...
8594     source Add To The Conversation Using Facebook ...
10095    source Add To The Conversation Using Facebook ...
10343    source Add To The Conversation Using Facebook ...
11975    source Add To The Conversation Using Facebook ...
12212    source Add To The Conversation Using Facebook ...
12615    source Add To The Conversation Using Facebook .

In [13]:
train = train[train['text'] != "source Add To The Conversation Using Facebook Comments"]

# Preprocessing

In [14]:
train['combined'] = train['title'] + ' ' + train['text']

In [15]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
def tokenize_text(text):
    return word_tokenize(text)

In [16]:
def preprocess(df):
    # Convert 'combined' column to strings
    df['combined'] = df['combined'].astype(str)
    
    # Lowercase
    df['combined'] = df['combined'].str.lower()

    # Expanding Contractions
    df['combined'] = df['combined'].apply(lambda x: contractions.fix(x))

    # Removing the URLs
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

    # Removing the HTML tags
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Removing the numbers
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'\d+', '', x))

    # Removing the extra whitespaces
    df['combined'] = df['combined'].apply(lambda x: x.strip())

    # Removing the punctuation
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Removing the stopwords
    df['combined'] = df['combined'].apply(remove_stopwords)

    # Tokenization
    df['combined'] = df['combined'].apply(tokenize_text)

    return df

In [18]:
train = preprocess(train)

In [19]:
train['combined'].dropna(inplace=True)

# Model

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Import model
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.model_selection import GridSearchCV, train_test_split, StratifiedKFold

In [21]:
X_train = train['combined']
y_train = train['label']

In [22]:
X_train.head()

0    [house, dem, aide, even, see, comeys, letter, ...
1    [flynn, hillary, clinton, big, woman, campus, ...
2    [truth, might, get, fired, truth, might, get, ...
3    [civilians, killed, single, us, airstrike, ide...
4    [iranian, woman, jailed, fictional, unpublishe...
Name: combined, dtype: object

In [23]:
vectorizer = TfidfVectorizer()
X_train = X_train.apply(' '.join)
X_train = vectorizer.fit_transform(X_train)

## Param tuning

In [38]:
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

# Define a pipeline combining a text feature extractor with a simple classifier
mnb = MultinomialNB()

# Define the grid parameters
params = [
    {
        
        'alpha': [0.0,0.5, 1.0],  # smoothing parameter
        'fit_prior': [True, False],  # learn class prior probabilities or not
    }
]

# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=10)

# Create the grid search object
grid_search = GridSearchCV(estimator=mnb, param_grid=params, cv=stratified_kfold, n_jobs=-1, refit=True, verbose=2)

# Fit the grid search object to the data to compute the optimal model
grid_search.fit(X_train, y_train)

# Print the best score and the best parameters
print("Best Score: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:629: FutureWarning: The default val

Best Score:  0.9206530967186947
Best Parameters:  {'alpha': 0.0, 'fit_prior': False}


/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:629: FutureWarning: The default value for `force_alpha` will change to `True` in 1.4. To suppress this warning, manually set the value of `force_alpha`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/naive_bayes.py:635: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10. Use `force_alpha=True` to keep alpha unchanged.
  warnings.warn(


In [39]:
## Train on full dataset

In [40]:
# Training performance
clf = grid_search.best_estimator_
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00     10387
           1       1.00      0.99      1.00     10269

    accuracy                           1.00     20656
   macro avg       1.00      1.00      1.00     20656
weighted avg       1.00      1.00      1.00     20656



# Submission

In [41]:
test['combined'] = test['title'] + ' ' + test['text']
test = preprocess(test)
X_test = test['combined']
X_test = X_test.apply(' '.join)
X_test = vectorizer.transform(X_test)
id_test = test['id']
y_test_pred = clf.predict(X_test)

In [42]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('nb_submission.csv', index=False)

[CV] END ..........................alpha=0.0, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.0, fit_prior=True; total time=   0.2s
[CV] END .........................alpha=0.0, fit_prior=False; total time=   0.2s
[CV] END .........................alpha=0.0, fit_prior=False; total time=   0.2s
[CV] END .........................alpha=0.0, fit_prior=False; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END .........................alpha=0.5, fit_prior=False; total time=   0.2s
[CV] END .........................alpha=0.5, fit_prior=False; total time=   0.2s
[CV] END ..........................alpha=1.0, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=1.0, fit_prior=True; total time=   0.2s
[CV] END ...................

In [37]:
from joblib import dump
dump(clf, 'svm_model.joblib')

['svm_model.joblib']

[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END .........................alpha=0.5, fit_prior=False; total time=   0.2s
[CV] END .........................alpha=0.5, fit_prior=False; total time=   0.2s
[CV] END ..........................alpha=1.0, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=1.0, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=1.0, fit_prior=True; total time=   0.2s
[CV] END .........................alpha=1.0, fit_prior=False; total time=   0.2s
[CV] END .........................alpha=1.0, fit_prior=False; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ..........................alpha=0.5, fit_prior=True; total time=   0.2s
[CV] END ...................